# Argumentation framework

In [1]:
import sys
import os

sys.path.append(os.path.join(
    os.getcwd(),
    "utils",
))

In [2]:
from utils.stable import gen as gen_stable
from utils.grounded import gen as gen_grounded

g_stable = gen_stable(10000, 100, 1000, 100, 1000, 100, 1000, .6)
g_stable.save("stable.txt")

# g_grounded = gen_grounded(500, .3)
# g_grounded.save("grounded.txt")

Initialised _ReproducibleRandom with seed 164577625177


In [ ]:
from utils.solve import solve
from utils.benchmark import benchmark
from utils.reprandom import rr

rep = 10

solvers = ["dpll", "minisat", "manysat", "glucose"]

times = {}

for solver in solvers:
    times[solver] = 0

problems = []
for _ in range(rep):
    for solver in solvers:
        problems.append(solver)

sorted(problems, key = lambda _: rr.random())

for problem in problems:
    times[problem] += benchmark(solve, "grounded.txt", "SE-ST", solvers=[problem])

for solver in solvers:
    times[solver] /= rep

times